In [1]:
pip install numpy pandas matplotlib seaborn opencv-python tensorflow keras scikit-learn albumentations


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 401.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/81.7 kB ? eta -:--:--
     ---------------------------------------- 81.7/81.7 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/67.4 kB ? eta -:--:--
     ---------------------------------------- 67.4/67.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/289.6 kB ? eta -:--:--
   ---------------------------------------  286.7/289.6 kB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 289.6/289.6 kB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
    --------------------------------------- 0.9/39.4 MB 18.9 MB/s eta 0:00:03
   - -------------------------------------- 1.4/39.4 MB 15.1 MB/s eta 0:00:03
   - ---------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\aryai\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, concatenate
from sklearn.model_selection import train_test_split
import os


In [15]:
import zipfile

zip_path = r"C:\Users\aryai\Downloads\archive.zip"  # Path to the ZIP file
extract_to = r"C:\Users\aryai\Downloads\archive"  # Folder to extract into

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete.")


Extraction complete.


In [21]:
# Define path to dataset
image_folder =r"C:\Users\aryai\Downloads\archive\image"
mask_folder = r"C:\Users\aryai\Downloads\archive\mask"

# Load images and masks
def load_images_masks(image_folder, mask_folder):
    images = []
    masks = []
    image_files = sorted(os.listdir(image_folder))
    mask_files = sorted(os.listdir(mask_folder))

    for img_file, mask_file in zip(image_files, mask_files):
        img = cv2.imread(os.path.join(image_folder, img_file), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(os.path.join(mask_folder, mask_file), cv2.IMREAD_GRAYSCALE)

        img = cv2.resize(img, (128, 128)) / 255.0  # Normalize images
        mask = cv2.resize(mask, (128, 128)) / 255.0

        images.append(img)
        masks.append(mask)

    return np.array(images).reshape(-1, 128, 128, 1), np.array(masks).reshape(-1, 128, 128, 1)

X, Y = load_images_masks(image_folder, mask_folder)


In [22]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


In [23]:
def unet_model(input_size=(128, 128, 1)):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D(pool_size=(2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D(pool_size=(2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u1 = UpSampling2D(size=(2, 2))(c3)
    u1 = concatenate([u1, c2], axis=-1)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D(size=(2, 2))(c4)
    u2 = concatenate([u2, c1], axis=-1)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=20, batch_size=16)


Epoch 1/20
 57/281 ━━━━━━━━━━━━━━━━━━━━ 8:50 2s/step - accuracy: 0.9877 - loss: 0.1697

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
index = np.random.randint(0, len(X_val))
pred = model.predict(np.expand_dims(X_val[index], axis=0))[0]

plt.figure(figsize=(10, 4))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(X_val[index].squeeze(), cmap='gray')

plt.subplot(1, 3, 2)
plt.title("True Mask")
plt.imshow(Y_val[index].squeeze(), cmap='gray')

plt.subplot(1, 3, 3)
plt.title("Predicted Mask")
plt.imshow(pred.squeeze(), cmap='gray')

plt.show()


In [ ]:
model.save("nerve_segmentation_unet.h5")


In [ ]:
loaded_model = keras.models.load_model("nerve_segmentation_unet.h5")
